In [ ]:
pip install datasets tensorflow scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.4 MB/s eta 0:00:00


In [ ]:
import datasets
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, LSTM, Dense, Dropout, SimpleRNN
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import numpy as np
import random
import os
import json
from google.colab import drive

drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/ShakespeareModels'
MODEL_DIR = os.path.join(BASE_DIR, "saved_models_gpu")
TOKENIZER_PATH = os.path.join(MODEL_DIR, "tokenizer.json")

os.makedirs(MODEL_DIR, exist_ok=True)

# Load dataset
data = datasets.load_dataset('tiny_shakespeare', split='train')
text_corpus = data['text']
raw_text = text_corpus[0]

print(f"Dataset: {data}")
print(f"First 50 characters of text: {raw_text[:50]}")

# Hyperparameter spaces
general_params = {
    'learning_rate': [1e-3, 1e-4, 1e-5],
    'batch_size': [32, 64, 128],
    'optimizer': ['adam', 'sgd', 'rmsprop'],
    'activation_function': ['relu', 'tanh', 'sigmoid'],
    'dropout_rate': [0.0, 0.2, 0.5],
    'weight_initialization': ['glorot_uniform', 'he_normal']
}

cnn_params = {
    'filters': [32, 64, 128],
    'kernels': [3, 5, 7],
    'strides': [1, 2],
    'layers': [1, 2]
}

rnn_params = {
    'units': [64, 128, 256],
    'layers': [1, 2, 3],
    'cell_type': ['lstm', 'simple_rnn']
}

tokenizer = Tokenizer(char_level=True, oov_token="<unk>")
tokenizer.fit_on_texts([raw_text])

with open(TOKENIZER_PATH, 'w', encoding='utf-8') as file:
    file.write(tokenizer.to_json())
print(f"Tokenizer saved to: {TOKENIZER_PATH}")

encoded_text = tokenizer.texts_to_sequences([raw_text])[0]
vocab_count = len(tokenizer.word_index) + 1

seq_length = 40
input_sequences, output_tokens = [], []
for idx in range(len(encoded_text) - seq_length):
    input_sequences.append(encoded_text[idx:idx + seq_length])
    output_tokens.append(encoded_text[idx + seq_length])

X_data = np.array(input_sequences)
y_data = np.array(output_tokens)

X_train, X_valid, y_train, y_valid = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

def create_cnn_model(params):
    model = Sequential()
    model.add(Embedding(vocab_count, params['filters'][0], input_length=seq_length))
    for i in range(params['layers']):
        model.add(Conv1D(filters=params['filters'][i],
                         kernel_size=params['kernels'][i],
                         strides=params['strides'][i],
                         activation=params['activation_function'],
                         kernel_initializer=params['weight_initialization']))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(params['dropout_rate']))
    model.add(Flatten())
    model.add(Dense(vocab_count, activation='softmax', kernel_initializer=params['weight_initialization']))
    return model

def create_rnn_model(params):
    model = Sequential()
    model.add(Embedding(vocab_count, params['units'][0], input_length=seq_length))
    for i in range(params['layers']):
        RNN_Cell = LSTM if params['cell_type'] == 'lstm' else SimpleRNN
        model.add(RNN_Cell(params['units'][i],
                           activation=params['activation_function'],
                           return_sequences=(i < params['layers'] - 1),
                           kernel_initializer=params['weight_initialization']))
        model.add(Dropout(params['dropout_rate']))
    model.add(Dense(vocab_count, activation='softmax', kernel_initializer=params['weight_initialization']))
    return model

def select_random_params(model_choice):
    config = {key: random.choice(val) for key, val in general_params.items()}
    if model_choice == 'cnn':
        layers = random.choice(cnn_params['layers'])
        config.update({
            'filters': [random.choice(cnn_params['filters']) for _ in range(layers)],
            'kernels': [random.choice(cnn_params['kernels']) for _ in range(layers)],
            'strides': [random.choice(cnn_params['strides']) for _ in range(layers)],
            'layers': layers
        })
    elif model_choice == 'rnn':
        layers = random.choice(rnn_params['layers'])
        config.update({
            'units': [random.choice(rnn_params['units']) for _ in range(layers)],
            'layers': layers,
            'cell_type': random.choice(rnn_params['cell_type'])
        })
    else:
        raise ValueError("Invalid model type.")
    return config

trials = 5
best_cnn_acc = -1.0
best_cnn_config = None
best_rnn_acc = -1.0
best_rnn_config = None

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

print("--- Starting CNN Hyperparameter Search ---")
for run in range(trials):
    print(f"\n--- CNN Trial {run+1}/{trials} ---")
    params = select_random_params('cnn')
    print("Hyperparameters:", params)

    cnn_model = create_cnn_model(params)
    optimizer_choice = {'adam': Adam, 'sgd': SGD, 'rmsprop': RMSprop}[params['optimizer']]
    cnn_model.compile(optimizer=optimizer_choice(learning_rate=params['learning_rate']),
                      loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = cnn_model.fit(X_train, y_train, epochs=5, batch_size=params['batch_size'],
                            validation_data=(X_valid, y_valid), callbacks=[early_stop], verbose=1)

    val_acc = history.history['val_accuracy'][-1]
    print(f"Validation Accuracy: {val_acc:.4f}")

    if val_acc > best_cnn_acc:
        best_cnn_acc = val_acc
        best_cnn_config = params
        cnn_save_path = os.path.join(MODEL_DIR, f"best_cnn_model_gpu_trial_{run+1}_val_acc_{val_acc:.4f}.keras")
        cnn_model.save(cnn_save_path)
        print(f"  --- Best CNN model saved to: {cnn_save_path}")

print("\n--- CNN Hyperparameter Search Completed ---")
print(f"Best CNN Validation Accuracy: {best_cnn_acc:.4f}")
print("Best CNN Hyperparameters:", best_cnn_config)

print("\n--- Starting RNN Hyperparameter Search ---")
for run in range(trials):
    print(f"\n--- RNN Trial {run+1}/{trials} ---")
    params = select_random_params('rnn')
    print("Hyperparameters:", params)

    rnn_model = create_rnn_model(params)
    optimizer_choice = {'adam': Adam, 'sgd': SGD, 'rmsprop': RMSprop}[params['optimizer']]
    rnn_model.compile(optimizer=optimizer_choice(learning_rate=params['learning_rate']),
                      loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    history = rnn_model.fit(X_train, y_train, epochs=5, batch_size=params['batch_size'],
                             validation_data=(X_valid, y_valid), callbacks=[early_stop], verbose=1)

    val_acc = history.history['val_accuracy'][-1]
    print(f"Validation Accuracy: {val_acc:.4f}")

    if val_acc > best_rnn_acc:
        best_rnn_acc = val_acc
        best_rnn_config = params
        rnn_save_path = os.path.join(MODEL_DIR, f"best_rnn_model_gpu_trial_{run+1}_val_acc_{val_acc:.4f}.keras")
        rnn_model.save(rnn_save_path)
        print(f"  --- Best RNN model saved to: {rnn_save_path}")

print("\n--- RNN Hyperparameter Search Completed ---")
print(f"Best RNN Validation Accuracy: {best_rnn_acc:.4f}")
print("Best RNN Hyperparameters:", best_rnn_config)


print("\n--- Discussion ---")
print("Hyperparameter search complete using GPU. Best CNN and RNN models and tokenizer are saved in 'saved_models_gpu/' directory.") # Changed directory name in discussion
print(f"Tokenizer saved to: {TOKENIZER_SAVE_PATH}")
print("Remember to evaluate these saved models on a dedicated test set for final performance assessment.")
print("Reduced trials and epochs for faster search. Increase them for a more thorough search.")

Mounted at /content/drive
Dataset: Dataset({
    features: ['text'],
    num_rows: 1
})
First 50 characters of text: First Citizen:
Before we proceed any further, hear
Tokenizer saved to: saved_models_gpu/tokenizer.json
--- Starting CNN Hyperparameter Search ---

--- CNN Trial 1/5 ---
Hyperparameters: {'learning_rate': 0.001, 'batch_size': 32, 'optimizer': 'sgd', 'activation_function': 'tanh', 'dropout_rate': 0.2, 'weight_initialization': 'he_normal', 'num_filters': [128], 'kernel_size': [7], 'stride': [1], 'num_cnn_layers': 1}
Epoch 1/5
25096/25096 ━━━━━━━━━━━━━━━━━━━━ 97s 4ms/step - accuracy: 0.1427 - loss: 3.2370 - val_accuracy: 0.1512 - val_loss: 3.0628
Epoch 2/5
25096/25096 ━━━━━━━━━━━━━━━━━━━━ 140s 4ms/step - accuracy: 0.1535 - loss: 3.0567 - val_accuracy: 0.1680 - val_loss: 3.0099
Epoch 3/5
25096/25096 ━━━━━━━━━━━━━━━━━━━━ 87s 3ms/step - accuracy: 0.1815 - loss: 2.9801 - val_accuracy: 0.1985 - val_loss: 2.8891
Epoch 4/5
25096/25096 ━━━━━━━━━━━━━━━━━━━━ 94s 4ms/step - accuracy: 0

In [ ]:
import os
import shutil
from google.colab import drive

def transfer_models_to_drive(target_drive_path):
    """
    Transfers the 'saved_models_gpu' directory from Colab's storage
    to a specified folder within Google Drive.

    Args:
        target_drive_path (str): Full path to the destination in Google Drive
                                 (e.g., "/content/drive/MyDrive/ProjectModels/saved_models_gpu").
                                 Ensure the desired final folder name is included.
    """

    # Step 1: Mount Google Drive
    try:
        drive.mount('/content/drive', force_remount=True)
        print(" Google Drive mounted successfully.")
    except Exception as error:
        print(f" Failed to mount Google Drive: {error}")
        return False

    # Step 2: Define source and destination paths
    COLAB_MODELS_DIR = "/content/saved_models_gpu"
    DRIVE_DESTINATION_DIR = target_drive_path

    # Verify if the source directory exists
    if not os.path.isdir(COLAB_MODELS_DIR):
        print(f" Source directory '{COLAB_MODELS_DIR}' not found.")
        print("Ensure your hyperparameter search has been executed and models are saved.")
        return False

    # Step 3: Perform the directory copy
    try:
        shutil.copytree(COLAB_MODELS_DIR, DRIVE_DESTINATION_DIR, dirs_exist_ok=True)
        print(f" Successfully copied models to '{DRIVE_DESTINATION_DIR}'.")
        return True
    except Exception as error:
        print(f" Error while copying: {error}")
        print("Verify the destination path and check Google Drive access.")
        return False


if __name__ == '__main__':
    # --- USER CONFIGURATION ---
    DRIVE_SAVE_PATH = "/content/drive/MyDrive/ShakespeareExperiments/saved_models_gpu"  # Update to your desired Google Drive folder
    # --- END USER CONFIGURATION ---

    if transfer_models_to_drive(DRIVE_SAVE_PATH):
        print("\n---  Model transfer complete. ---")
        print(f"Your models are now available at: '{DRIVE_SAVE_PATH}'")
    else:
        print("\n---  Model transfer failed. Please review the errors above. ---")


Mounted at /content/drive
Google Drive mounted successfully.
Successfully copied '/content/saved_models_gpu' to '/content/drive/MyDrive/ShakespeareModels/saved_models_gpu' in Google Drive.

--- Copying process completed. ---
You can now find your 'saved_models_gpu' directory in Google Drive at: '/content/drive/MyDrive/ShakespeareModels/saved_models_gpu'


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import tokenizer_from_json
import datasets
import numpy as np
import os


def evaluate_shakespeare_models(drive_models_path):
    """
    Loads and evaluates the best saved CNN and RNN models on the test split
    of the tiny_shakespeare dataset.

    Assumes both models and the tokenizer are stored in Google Drive under the provided path.

    Args:
        drive_models_path (str): Full Google Drive path to the directory containing
                                 the models and tokenizer (e.g., "/content/drive/MyDrive/ShakespeareModels").
    """

    # Step 1: Mount Google Drive if in Colab
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        print("Google Drive mounted.")
    except ImportError:
        print("Not running inside Colab. Skipping Google Drive mount.")
    except Exception as e:
        print(f"Error mounting Google Drive: {e}")
        return

    # Step 2: Setup paths
    MODELS_DIR = os.path.join(drive_models_path, "saved_models_gpu")
    TOKENIZER_PATH = os.path.join(MODELS_DIR, "tokenizer.json")
    CNN_MODEL_PATH = os.path.join(MODELS_DIR, "best_cnn_model_gpu_trial_5_val_acc_0.2284.keras")
    RNN_MODEL_PATH = os.path.join(MODELS_DIR, "best_rnn_model_gpu_trial_3_val_acc_0.5457.keras")
    SEQ_LENGTH = 40

    # Step 3: Load tokenizer
    print("Loading tokenizer...")
    try:
        with open(TOKENIZER_PATH, 'r', encoding='utf-8') as f:
            tokenizer = tokenizer_from_json(f.read())
        vocab_size = len(tokenizer.word_index) + 1
        print(f"Tokenizer loaded. Vocabulary size: {vocab_size}")
    except FileNotFoundError:
        print(f"Tokenizer not found at '{TOKENIZER_PATH}'.")
        return

    # Step 4: Load test dataset
    print("Loading test data...")
    test_data = datasets.load_dataset('tiny_shakespeare', split='test')
    test_text = test_data['text'][0]
    print("Test dataset loaded.")

    # Step 5: Preprocess test data
    print("Preprocessing test data...")
    tokenized_text = tokenizer.texts_to_sequences([test_text])[0]

    X_test, y_test = [], []
    for i in range(len(tokenized_text) - SEQ_LENGTH):
        X_test.append(tokenized_text[i:i + SEQ_LENGTH])
        y_test.append(tokenized_text[i + SEQ_LENGTH])

    X_test = np.array(X_test)
    y_test = np.array(y_test)
    print(f"Prepared {len(X_test)} test sequences.")

    # Step 6: Load models
    print("Loading CNN model...")
    try:
        cnn_model = tf.keras.models.load_model(CNN_MODEL_PATH)
        print("CNN model loaded.")
    except FileNotFoundError:
        print(f"CNN model not found at '{CNN_MODEL_PATH}'.")
        return

    print("Loading RNN model...")
    try:
        rnn_model = tf.keras.models.load_model(RNN_MODEL_PATH)
        print("RNN model loaded.")
    except FileNotFoundError:
        print(f"RNN model not found at '{RNN_MODEL_PATH}'.")
        return

    # Step 7: Evaluate models
    print("\nEvaluating CNN model...")
    cnn_loss, cnn_accuracy = cnn_model.evaluate(X_test, y_test, verbose=1)
    print(f"CNN Test Loss: {cnn_loss:.4f}, CNN Test Accuracy: {cnn_accuracy:.4f}")

    print("\nEvaluating RNN model...")
    rnn_loss, rnn_accuracy = rnn_model.evaluate(X_test, y_test, verbose=1)
    print(f"RNN Test Loss: {rnn_loss:.4f}, RNN Test Accuracy: {rnn_accuracy:.4f}")

    print("\nEvaluation complete.")


if __name__ == '__main__':
    # --- USER CONFIGURATION ---
    DRIVE_MODELS_FOLDER = "/content/drive/MyDrive/ShakespeareModels"
    # --- END USER CONFIGURATION ---

    evaluate_shakespeare_models(DRIVE_MODELS_FOLDER)


Mounted at /content/drive
Google Drive mounted successfully.
Loading tokenizer...
Tokenizer loaded. Vocabulary size: 41
Loading test dataset...
Test dataset loaded.
Preprocessing test data...
Total test patterns: 55730
Test data preprocessed.
Loading saved CNN model...
CNN model loaded.
Loading saved RNN model...
RNN model loaded.

--- Evaluating CNN Model ---
1742/1742 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.2252 - loss: 2.7862
CNN Test Loss: 2.8050, CNN Test Accuracy: 0.2217

--- Evaluating RNN Model ---
1742/1742 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.5144 - loss: 1.5774
RNN Test Loss: 1.6520, RNN Test Accuracy: 0.5017

--- Testing Completed ---
Test results are printed above.


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
import os


def create_text(model, tokenizer, start_text, length=500, temp=1.0):
    """
    Generates text from a trained RNN model using a given seed.

    Args:
        model: Trained Keras RNN model.
        tokenizer: Tokenizer used during training.
        start_text: Text to start the generation from.
        length: Number of characters to generate.
        temp: Temperature for randomness in predictions.

    Returns:
        The generated text string.
    """

    output_text = start_text
    sequence = tokenizer.texts_to_sequences([start_text])[0]
    sequence = pad_sequences([sequence], maxlen=40, padding='pre')

    for _ in range(length):
        predictions = model.predict(sequence, verbose=0)
        next_index = pick_prediction(predictions[0], temp)
        next_char = tokenizer.index_word.get(next_index)

        if next_char is None:
            break

        output_text += next_char
        sequence = np.append(sequence[:, 1:], [[next_index]], axis=1)

    return output_text


def pick_prediction(pred_probs, temp=1.0):
    """
    Selects an index from the prediction probabilities using temperature.
    """
    pred_probs = np.asarray(pred_probs).astype('float64')
    pred_probs = np.log(pred_probs + 1e-8) / temp
    exp_preds = np.exp(pred_probs)
    prob_dist = exp_preds / np.sum(exp_preds)
    return np.argmax(np.random.multinomial(1, prob_dist, 1))


def load_resources(drive_path):
    """
    Loads the tokenizer and the RNN model from Google Drive.
    """
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
        print("Drive mounted successfully.")
    except ImportError:
        print("This environment is not Colab.")
    except Exception as err:
        print(f"Drive mount failed: {err}")
        return None, None

    models_folder = os.path.join(drive_path, "saved_models_gpu")
    tokenizer_file = os.path.join(models_folder, "tokenizer.json")
    model_file = os.path.join(models_folder, "best_rnn_model_gpu_trial_3_val_acc_0.5457.keras")

    print("Loading tokenizer...")
    try:
        with open(tokenizer_file, 'r', encoding='utf-8') as file:
            tokenizer_data = file.read()
            tokenizer = tokenizer_from_json(tokenizer_data)
        print("Tokenizer loaded.")
    except FileNotFoundError:
        print(f"Tokenizer not found at: {tokenizer_file}")
        return None, None

    print("Loading RNN model...")
    try:
        rnn_model = tf.keras.models.load_model(model_file)
        print("RNN model loaded.")
        return tokenizer, rnn_model
    except FileNotFoundError:
        print(f"Model file not found at: {model_file}")
        return None, None


if __name__ == '__main__':
    DRIVE_PATH = "/content/drive/MyDrive/ShakespeareModels"
    INITIAL_TEXT = "ROMEO:"
    CHARACTERS_TO_GENERATE = 500
    TEMPERATURE_VALUE = 0.8

    tokenizer, model = load_resources(DRIVE_PATH)

    if tokenizer and model:
        print(f"\nGenerating Shakespearen Text {TEMPERATURE_VALUE}...\n")
        result_text = create_text(
            model, tokenizer, INITIAL_TEXT, CHARACTERS_TO_GENERATE, TEMPERATURE_VALUE
        )
        print(result_text)
    else:
        print("Tokenizer or model could not be loaded. Please check file paths.")


Mounted at /content/drive
Google Drive mounted successfully.
Loading tokenizer...
Tokenizer loaded.
Loading RNN model...
RNN model loaded.

--- Generating Shakespearean Text (Temperature: 0.8) ---
ROMEO:
we here at my life.

king richard iii:
master man.

mariana:
when all him?

remeonere:
how thou shalt upon the mercy to feel
shall we know, steed and fear age and but well them,
and i can a not to well; so shall sir
accure nor him.

provost:
'tis honed, and that have be grace of pave,
i can more may shall for the love to it
this your profent and forch'd upon the hold
to stall and i know, mean, what we have being kill.

menenius:
wherefore you have our sarge-tont it;
that defation were course th
